In [37]:
import os
import csv
from collections import defaultdict
import json

# To finetune

In [225]:
DATASET_VAR = 'augm_wabad'
DATASET_NAME = 'dataset'
RESULTS_PATH = f'/home/giacomoschiavo/finetuning-BirdNET/models/finetuned_1/{DATASET_VAR}'
VALID_TABLE = f'{RESULTS_PATH}/valid/BirdNET_SelectionTable.txt'
TRAIN_PATH = f'/home/giacomoschiavo/segments/{DATASET_NAME}/train'
VALID_PATH = f'/home/giacomoschiavo/segments/{DATASET_NAME}/valid'
TEST_PATH = f'/home/giacomoschiavo/segments/{DATASET_NAME}/test'
TEST_TABLE = f'{RESULTS_PATH}/test/BirdNET_SelectionTable.txt'

In [226]:
print(f'python -m birdnet_analyzer.train --i {TRAIN_PATH} --o {RESULTS_PATH}/{DATASET_VAR}.tflite --batch_size 64 --threads 16 --val_split 0.01 --epochs 150 --mixup --cache_mode save --cache_file {RESULTS_PATH}/{DATASET_VAR}.npz')

python -m birdnet_analyzer.train --i /home/giacomoschiavo/segments/dataset/train --o /home/giacomoschiavo/finetuning-BirdNET/models/finetuned_1/augm_wabad/augm_wabad.tflite --batch_size 64 --threads 16 --val_split 0.01 --epochs 150 --mixup --cache_mode save --cache_file /home/giacomoschiavo/finetuning-BirdNET/models/finetuned_1/augm_wabad/augm_wabad.npz


# To analize

In [227]:
MIN_CONF = '0.05'

In [228]:
print(f'python -m birdnet_analyzer.analyze --i {VALID_PATH} --o {RESULTS_PATH}/valid --slist /home/giacomoschiavo/finetuning-BirdNET/models/finetuned_1/Labels.txt --threads 16 --combine_results --min_conf {MIN_CONF} --classifier {RESULTS_PATH}/{DATASET_VAR}.tflite')

python -m birdnet_analyzer.analyze --i /home/giacomoschiavo/segments/dataset/valid --o /home/giacomoschiavo/finetuning-BirdNET/models/finetuned_1/augm_wabad/valid --slist /home/giacomoschiavo/finetuning-BirdNET/models/finetuned_1/Labels.txt --threads 16 --combine_results --min_conf 0.05 --classifier /home/giacomoschiavo/finetuning-BirdNET/models/finetuned_1/augm_wabad/augm_wabad.tflite


In [229]:
print(f'python -m birdnet_analyzer.analyze --i {TEST_PATH} --o {RESULTS_PATH}/test --slist /home/giacomoschiavo/finetuning-BirdNET/models/finetuned_1/Labels.txt --threads 16 --combine_results --min_conf {MIN_CONF} --classifier {RESULTS_PATH}/{DATASET_VAR}.tflite')

python -m birdnet_analyzer.analyze --i /home/giacomoschiavo/segments/dataset/test --o /home/giacomoschiavo/finetuning-BirdNET/models/finetuned_1/augm_wabad/test --slist /home/giacomoschiavo/finetuning-BirdNET/models/finetuned_1/Labels.txt --threads 16 --combine_results --min_conf 0.05 --classifier /home/giacomoschiavo/finetuning-BirdNET/models/finetuned_1/augm_wabad/augm_wabad.tflite


# Analysis

In [230]:
with open('/home/giacomoschiavo/finetuning-BirdNET/utils/species_dict_map.json') as f:
    species_dict = json.load(f)
inv_species_dict = {value: key for key, value in species_dict.items()}

In [231]:
def get_conf_scores():
    conf_scores = {}

    #  {'Aeroplane': [(np.float32(0.0), False),
    #               (np.float32(1.3937646e-30), False),
    #               (np.float32(1.0654355e-25), False),
    #               (np.float32(0.0), False),

    with open(VALID_TABLE, 'r') as f:
        reader = csv.DictReader(f, delimiter='\t')  # Usa TAB come separatore
        for row in reader:
            file_path = row['Begin Path']
            pred_species_name = row['Common Name']
            if row['Common Name'] in inv_species_dict:
                pred_species_name = '_'.join([inv_species_dict[row['Common Name']], row['Common Name']])
            true_species_name = file_path.split('/')[-2]
            confidence = float(row['Confidence'])
            if pred_species_name not in conf_scores:
                conf_scores[pred_species_name] = []
            is_correct = pred_species_name == true_species_name
            conf_scores[pred_species_name].append((confidence, is_correct))
    return conf_scores
            

In [232]:
conf_scores = get_conf_scores()

In [259]:
from sklearn.metrics import f1_score
import numpy as np 

DEFAULT_THRESH = 0.25

def compute_best_thresholds(conf_scores, num_thresholds=200, min_thresh=0.001, max_thresh=0.95):
    thresholds = {}

    for species, values in conf_scores.items():
        probs, truths = zip(*values)
        probs = np.array(probs)
        truths = np.array(truths).astype(int)

        best_thresh = DEFAULT_THRESH
        best_f1 = 0.0

        for thresh in np.linspace(min_thresh, max_thresh, num_thresholds):
            preds = (probs >= thresh).astype(int)
            f1 = f1_score(truths, preds, zero_division=0)
            if f1 > best_f1:
                best_f1 = f1
                best_thresh = thresh

        thresholds[species] = best_thresh
        print(f"📊 {species} -> {best_thresh:.3f}, F1-score: {best_f1:.3f}")

    return thresholds



In [260]:
best_thresholds = compute_best_thresholds(conf_scores)

📊 Sylvia atricapilla_Eurasian Blackcap -> 0.001, F1-score: 0.027
📊 Aeroplane -> 0.073, F1-score: 0.522
📊 Troglodytes troglodytes_Eurasian Wren -> 0.211, F1-score: 0.029
📊 Phylloscopus collybita_Common Chiffchaff -> 0.282, F1-score: 0.453
📊 Dendrocopos major_Great Spotted Woodpecker -> 0.292, F1-score: 0.455
📊 Regulus regulus_Goldcrest -> 0.053, F1-score: 0.036
📊 None -> 0.053, F1-score: 0.705
📊 Certhia familiaris_Eurasian Treecreeper -> 0.163, F1-score: 0.019
📊 Fringilla coelebs_Common Chaffinch -> 0.001, F1-score: 0.289
📊 Regulus ignicapilla_Common Firecrest -> 0.001, F1-score: 0.135
📊 Dryocopus martius_Black Woodpecker -> 0.344, F1-score: 0.778
📊 Turdus philomelos_Song Thrush -> 0.259, F1-score: 0.592
📊 Erithacus rubecula_European Robin -> 0.053, F1-score: 0.011
📊 Turdus merula_Eurasian Blackbird -> 0.125, F1-score: 0.012
📊 Periparus ater_Coal Tit -> 0.211, F1-score: 0.019
📊 Loxia curvirostra_Common Crossbill -> 0.306, F1-score: 0.875
📊 Lophophanes cristatus_Crested Tit -> 0.645, F1-

In [261]:
with open(f'/home/giacomoschiavo/finetuning-BirdNET/utils/{DATASET_NAME}/true_segments_test.json') as f:
    true_segments_test = json.load(f)

In [262]:
with open(f'/home/giacomoschiavo/finetuning-BirdNET/utils/{DATASET_NAME}/true_segments_train.json') as f:
    true_segments_full = json.load(f)

In [263]:
true_segments_full.update(true_segments_test)

In [264]:
pred_segments = {}
#  {'Aeroplane': [(np.float32(0.0), False),
#               (np.float32(1.3937646e-30), False),
#               (np.float32(1.0654355e-25), False),
#               (np.float32(0.0), False),

with open(TEST_TABLE, 'r') as f:
    reader = csv.DictReader(f, delimiter='\t')  # Usa TAB come separatore
    for row in reader:
        file_path = row['Begin Path']
        audio_name = os.path.basename(file_path)    # in test non sono presenti WABAD o augm
        only_audio_name = "_".join(audio_name.split("_")[:2]) + ".WAV"
        segm = "_".join(audio_name.split(".")[0].split("_")[-2:])
        pred_species_name = row['Common Name']
        if row['Common Name'] in inv_species_dict:
            pred_species_name = '_'.join([inv_species_dict[row['Common Name']], row['Common Name']])
        # if len(file_path.split('/')[-2].split('_')) == 1:       # skip if None, Pecking, etc. 
        #     continue
        confidence = float(row['Confidence'])
        if only_audio_name not in pred_segments:
            pred_segments[only_audio_name] = {}
        if pred_species_name not in best_thresholds:
            best_thresholds[pred_species_name] = DEFAULT_THRESH
        if confidence >= best_thresholds[pred_species_name]:
            if segm not in pred_segments[only_audio_name]:
                pred_segments[only_audio_name][segm] = set()
            if "None" in pred_segments[only_audio_name][segm]:
                continue
            if pred_species_name == "None":
                pred_segments[only_audio_name][segm] = set(["None"])
            else:
                pred_segments[only_audio_name][segm].add(pred_species_name)
            

In [265]:
from sklearn.preprocessing import MultiLabelBinarizer

test_species_list = list(best_thresholds.keys())
mlb = MultiLabelBinarizer()
mlb.fit([test_species_list])

len(mlb.classes_)

22

In [266]:
true_segments = defaultdict(dict)

for species in os.listdir(TEST_PATH):
    if species not in test_species_list:
        print(species, 'ignored')
        continue
    for audio in os.listdir(os.path.join(TEST_PATH, species)):
        audio = audio.split('.')[0]
        date, time, segm1, segm2 = audio.split('_')
        audio_name = '_'.join([date, time]) + '.WAV'
        segm = '_'.join([segm1, segm2])
        if segm not in true_segments[audio_name]:
            true_segments[audio_name][segm] = []
        true_segments[audio_name][segm].extend([species])

In [267]:
for audio in true_segments.keys():
    if audio not in pred_segments:
        pred_segments[audio] = {}
    for segm in true_segments[audio].keys():
        if segm not in pred_segments[audio]:
            pred_segments[audio][segm] = []
        

In [268]:
y_pred = []
y_true = []
for audio in pred_segments:
    # sort in increasing order
    sortable_true_segments = { str(key): value for key, value in true_segments[audio].items() }
    sortable_pred_segments = { str(key): value for key, value in pred_segments[audio].items() }
    sorted_true_segments = dict(sorted(sortable_true_segments.items()))
    sorted_pred_segments = dict(sorted(sortable_pred_segments.items()))
    y_true.append(mlb.transform(sorted_true_segments.values()))     # apply transform on every label of every segment
    y_pred.append(mlb.transform(sorted_pred_segments.values()))     # apply transform on every label of every segment

y_true = np.vstack(y_true)
y_pred = np.vstack(y_pred)

In [269]:
from sklearn.metrics import classification_report
import pandas as pd
report = classification_report(y_true, y_pred, target_names=mlb.classes_, zero_division=0, output_dict=True)

report_df = pd.DataFrame(report).T
report_df

,precision,recall,f1-score,support
Aeroplane,0.000000,0.000000,0.000000,22.0
Certhia familiaris_Eurasian Treecreeper,0.024390,0.333333,0.045455,3.0
Dendrocopos major_Great Spotted Woodpecker,0.000000,0.000000,0.000000,16.0
Dryocopus martius_Black Woodpecker,0.000000,0.000000,0.000000,21.0
Erithacus rubecula_European Robin,0.713253,0.539162,0.614108,549.0
Fringilla coelebs_Common Chaffinch,0.516539,0.827727,0.636114,981.0
Lophophanes cristatus_Crested Tit,0.000000,0.000000,0.000000,3.0
Loxia curvirostra_Common Crossbill,0.500000,0.061224,0.109091,49.0
Muscicapa striata_Spotted Flycatcher,0.000000,0.000000,0.000000,14.0
None,0.882547,0.888585,0.885556,5116.0


In [270]:
with open(f'{RESULTS_PATH}/classification_report.json', 'w') as f:
    json.dump(report, f)

In [271]:
from sklearn.metrics import multilabel_confusion_matrix

mcm = multilabel_confusion_matrix(y_true, y_pred)

# Per visualizzarle in un DataFrame leggibile:
mlb_labels = mlb.classes_  # nome delle classi, già usato da te

# Costruisco un dataframe
conf_matrices = []
for idx, label in enumerate(mlb_labels):
    tn, fp, fn, tp = mcm[idx].ravel()
    conf_matrices.append({
        'Class': label,
        'TP': tp,
        'FP': fp,
        'FN': fn,
        'TN': tn
    })

conf_df = pd.DataFrame(conf_matrices)
conf_df


,Class,TP,FP,FN,TN
0,Aeroplane,0,1,22,8105
1,Certhia familiaris_Eurasian Treecreeper,1,40,2,8085
2,Dendrocopos major_Great Spotted Woodpecker,0,0,16,8112
3,Dryocopus martius_Black Woodpecker,0,0,21,8107
4,Erithacus rubecula_European Robin,296,119,253,7460
5,Fringilla coelebs_Common Chaffinch,812,760,169,6387
6,Lophophanes cristatus_Crested Tit,0,9,3,8116
7,Loxia curvirostra_Common Crossbill,3,3,46,8076
8,Muscicapa striata_Spotted Flycatcher,0,7,14,8107
9,None,4546,605,570,2407
